In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sps
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import sys
sys.path.append("/Users/alessiorussointroito/Documents/GitHub/Structural-Perturbation-Method")

#from SPM_fast import SPM
from BiSPM import BiSPM

import similaripy as sim

In [2]:
n_svs = 300
k = 10
p = 0.3

In [3]:
df = pd.read_csv(
    "/Users/alessiorussointroito/Downloads/Telegram Desktop/recommender-system-2020-challenge-polimi/data_train.csv")

le = LabelEncoder()
df['new_col'] = le.fit_transform(df.col)

row_size = len(df.row.unique())
col_size = len(le.classes_)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df.row, df.new_col, test_size=0.20, random_state=3)

In [5]:
bip_adj = sps.csr_matrix((np.ones(X_train.shape[0]), (X_train, y_train)), shape=(row_size, col_size))

In [6]:
test_indices = X_test.unique()
test_indices = np.sort(test_indices)
test = pd.DataFrame({'row': X_test, 'target': y_test})

targets = test.groupby(test.row)['target'].apply(lambda x: list(x))

In [7]:
urm = sim.normalization.bm25(bip_adj)

In [8]:
bspm = BiSPM(urm, target=test_indices, n_sv=n_svs, p=p)

In [9]:
rankings = bspm.k_runs(k=k)

SVD:  10%|█         | 1/10 [01:20<12:04, 80.52s/it]                  


KeyboardInterrupt: 

In [23]:
# Evaluation

def precision(is_relevant, relevant_items):
    # is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)
    return precision_score


def recall(is_relevant, relevant_items):
    # is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    recall_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)
    return recall_score


def MAP(is_relevant, relevant_items):
    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))
    map_score = np.sum(p_at_k) / np.min([len(relevant_items), len(is_relevant)])
    return map_score

In [24]:
n_users = len(test_indices)

cumulative_precision = 0.0
cumulative_recall = 0.0
cumulative_MAP = 0.0
num_eval = 0

at = 10

for i, user_id in enumerate(tqdm(test_indices)):
    relevant_items = targets[user_id]
    #recommended_items = rankings[user_id].argsort()[::-1][:at]
    recommended_items = rankings[i]

    # Filter Seen:
    # 1. Remove items already seen by the user
    seen_indices = sps.find(bip_adj[user_id])[1]    # Con [1] Prendiamo solo le colonne d'interesse della matrice di adiacenza
    mask = np.zeros(bip_adj.shape[1], dtype=bool)
    mask[seen_indices] = True
    
    recommended_items[mask] = -np.inf

    # Recommend
    recommended_items = recommended_items.argsort()[::-1][:at]

    num_eval += 1

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    cumulative_precision += precision(is_relevant, relevant_items)
    cumulative_recall += recall(is_relevant, relevant_items)
    cumulative_MAP += MAP(is_relevant, relevant_items)

cumulative_precision /= num_eval
cumulative_recall /= num_eval
cumulative_MAP /= num_eval

print(f"SPM n_eigen = {n_svs} , k = {k}  \n Precision = {cumulative_precision} \n Recall = {cumulative_recall} \n MAP = {cumulative_MAP}")

100%|██████████| 5644/5644 [00:14<00:00, 391.41it/s]

SPM n_eigen = 300 , k = 10  
 Precision = 0.026009922041105162 
 Recall = 0.026009922041105162 
 MAP = 0.04137939172768613


In [2]:
a = np.array([True, True, False, True])

In [3]:
np.cumsum(a)

array([1, 2, 2, 3])